In [ ]:
#importing all the packages
import cv2
import numpy as np
import torchvision.transforms as transforms
import time
import random
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
import torchvision
from torchvision import transforms, datasets, models, utils
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.nn import functional as F
from skimage import io, transform
from torch.optim import lr_scheduler
from skimage.transform import AffineTransform, warp
from tqdm import tqdm
device = torch.device('cuda')
import albumentations as A
from torch.cuda.amp import autocast, GradScaler

ModuleNotFoundError: ignored

In [ ]:
#create dataset class to return each batch
from matplotlib import pyplot as plt
import albumentations as A
import cv2
import pandas as pd
from torch.utils.data import Dataset
import torchvision.transforms as transforms


class SurfAutoencoder(Dataset):
    def __init__(self, csv_file, augment=False):
        self.annotations = pd.read_csv(csv_file)
        self.augment = augment
        self.transform = transforms.ToTensor()

        if self.augment:
            self.augmentation_transform = A.Compose([
                A.Rotate(limit = 90, p=1)
            ])
        else:
            self.augmentation_transform = None

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        first_image = cv2.imread(self.annotations.iloc[index, 0])
        if self.augmentation_transform is not None:
            augmented = self.augmentation_transform(image=first_image)
            first_image = augmented["image"]

        first_image = self.transform(first_image)

        return first_image


In [ ]:

train = SurfAutoencoder(csv_file = 'replace-with-training-.csv',augment = True)  #todo create csv file and put path here train.csv, val.csv
val = SurfAutoencoder(csv_file = 'replace-with-validation-.csv',augment = False)
test = SurfAutoencoder(csv_file = 'replace-with-test-.csv',augment = False)

batch_size = 16

train_loader = DataLoader(dataset = train, batch_size = batch_size,shuffle = True)
val_loader =DataLoader(dataset =  val, batch_size = batch_size,shuffle = False)
test_loader =DataLoader(dataset =  test, batch_size = batch_size,shuffle = False)


print(train.__getitem__(0))

In [ ]:
import torch
import torch.nn as nn

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 8, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 4, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(4, 8, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

input = torch.randn(1, 3, 224, 224)
model = Autoencoder()
output = model(input)
print(output.shape)


torch.Size([1, 3, 224, 224])


In [ ]:
num_epochs = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
autoencoder = Autoencoder()
autoencoder.to(device)

learning_rate = 0.001
optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

for epoch in range(num_epochs):
    autoencoder.train()
    running_loss = 0.0

    for images, _ in train_loader:
        images = images.to(device)
        reconstructions = autoencoder(images)
        loss = criterion(reconstructions, images)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)
    autoencoder.eval()
    val_loss = 0.0

    with torch.no_grad():
        for images, _ in val_loader:
            images = images.to(device)
            reconstructions = autoencoder(images)
            loss = criterion(reconstructions, images)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Validation Loss: {val_loss:.4f}")

autoencoder.eval()
test_loss = 0.0
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        reconstructions = autoencoder(images)
        loss = criterion(reconstructions, images)
        test_loss += loss.item()

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")
